# Data Prep of Chicago Food Inspections Data
This notebook reads in the ...

### References
- Data Source: https://healthdata.gov/dataset/food-inspections
- License: http://opendefinition.org/licenses/odc-odbl/

### Set Global Seed

In [1]:
SEED = 666

### Imports

In [2]:
import pandas as pd

### Read Chicago Food Inspections Data
Count records and columns.

In [3]:
food_inspections_df = pd.read_csv('../data/Food_Inspections.gz', compression='gzip')

In [4]:
food_inspections_df.shape

(195116, 17)

### Rename Columns

In [5]:
food_inspections_df.columns.tolist()

['Inspection ID',
 'DBA Name',
 'AKA Name',
 'License #',
 'Facility Type',
 'Risk',
 'Address',
 'City',
 'State',
 'Zip',
 'Inspection Date',
 'Inspection Type',
 'Results',
 'Violations',
 'Latitude',
 'Longitude',
 'Location']

In [6]:
columns = ['inspection_id', 'dba_name', 'aka_name', 'license_number', 'facility_type',
           'risk', 'address', 'city', 'state', 'zip', 'inspection_date', 'inspection_type',
           'result', 'violation', 'latitude', 'longitude', 'location']

In [7]:
food_inspections_df.columns = columns

### Convert Zip Code to String
And take only the first five digits, chopping off the decimal from reading the column as a float.

In [8]:
food_inspections_df['zip'] = food_inspections_df['zip'].astype(str).apply(lambda x: x.split('.')[0])

### Normalize Casing of Chicago
Accept only proper spellings of the word Chicago with mixed casing accepted

In [9]:
food_inspections_df['city'] = food_inspections_df['city'].apply(lambda x: 'CHICAGO'
                                                                if str(x).upper() == 'CHICAGO'
                                                                else x)

### Filter for Facilities in Chicago Illinois

In [10]:
loc_condition = (food_inspections_df['city'] == 'CHICAGO') & (food_inspections_df['state'] == 'IL')

### Drop Redundant Information
- Only Chicago is considered
- Only Illinois is considered
- Location is encoded as separate latitute and longitude columns

In [11]:
food_inspections_df = food_inspections_df[loc_condition].drop(['city', 'state', 'location'], 1)

In [12]:
food_inspections_df.shape

(194684, 14)

### Create Codes Corresponding to Each Violation Type by Parsing Violation Text

In [13]:
def create_violation_code(violation_text):
    
    if violation_text != violation_text:
        return -1
    else:
        return int(violation_text.split('.')[0])

In [14]:
food_inspections_df['violation_code'] = food_inspections_df['violation'].apply(create_violation_code)

### Create Attribute Dataframes with the Unique Inspection ID for Lookups if Needed
- Names
- Licenses
- Locations
- Violations
- Dates

In [15]:
names = ['inspection_id', 'dba_name', 'aka_name']
names_df = food_inspections_df[names]

In [16]:
licenses = ['inspection_id', 'license_number']
licenses_df = food_inspections_df[licenses]

In [17]:
locations = ['inspection_id', 'address', 'latitude', 'longitude']
locations_df = food_inspections_df[locations]

In [18]:
violations = ['inspection_id', 'violation', 'violation_code']
violations_df = food_inspections_df[violations]

In [19]:
dates = ['inspection_id', 'inspection_date']
dates_df = food_inspections_df[dates]

### Drop Features Not Used in Statistical Analysis
Features such as:

- `DBA Name`
- `AKA Name`
- `License #`
- `Address`
- `Violations`
- `Inspection Date`

May be examined following statistical analysis by joining on `Inspection ID`.  **Note:** future iterations of this work may wish to consider:

- Text from the the facility name
- Street level information from the facility address
- Prior inspections of the same facility by performing a temporal analysis of the data using `Inspection Date`

In [20]:
not_considered = ['dba_name', 'aka_name', 'license_number', 'address', 'violation', 'inspection_date']
food_inspections_df = food_inspections_df.drop(not_considered, 1)

### Create Dataframes of Count and Prevalence for Categorical Features
- Facility types
- Violation codes
- Zip codes
- Inspection types

In [21]:
facilities = food_inspections_df['facility_type'].value_counts()
facilities_df = pd.DataFrame({'facility_type':facilities.index, 'count':facilities.values})
facilities_df['prevalence'] = facilities_df['count'] / food_inspections_df.shape[0]

In [22]:
facilities_df.nlargest(10, 'count')

,facility_type,count,prevalence
0,Restaurant,129938,0.667430
1,Grocery Store,24829,0.127535
2,School,12062,0.061957
3,Children's Services Facility,3031,0.015569
4,Bakery,2837,0.014572
5,Daycare (2 - 6 Years),2682,0.013776
6,Daycare Above and Under 2 Years,2355,0.012097
7,Long Term Care,1340,0.006883
8,Catering,1190,0.006112
9,Liquor,847,0.004351


In [23]:
facilities_df.nsmallest(10, 'count')

,facility_type,count,prevalence
401,Herbalife Nutrition,1,0.000005
402,MOBILE DESSERT CART,1,0.000005
403,TAVERN/RESTAURANT,1,0.000005
404,PEDDLER,1,0.000005
405,NON-FOR PROFIT BASEMENT KIT,1,0.000005
406,PREPACAKAGED FOODS,1,0.000005
407,ALTERNATIVE SCHOOL,1,0.000005
408,REPACKAGING PLANT,1,0.000005
409,WHOLESALE & RETAIL,1,0.000005
410,ASSISTED LIVING FACILITY,1,0.000005


In [24]:
violations = food_inspections_df['violation_code'].value_counts()
violations_df = pd.DataFrame({'violation_code':violations.index, 'count':violations.values})
violations_df['prevalence'] = violations_df['count'] / food_inspections_df.shape[0]

In [25]:
violations_df.nlargest(10, 'count')

,violation_code,count,prevalence
0,-1,51543,0.264752
1,32,25408,0.130509
2,33,17449,0.089627
3,3,13609,0.069903
4,34,10576,0.054324
5,18,9989,0.051309
6,30,9441,0.048494
7,2,7166,0.036808
8,35,5631,0.028924
9,21,4582,0.023536


In [26]:
violations_df.nsmallest(10, 'count')

,violation_code,count,prevalence
61,61,1,0.000005
62,63,1,0.000005
60,70,6,0.000031
58,15,11,0.000057
59,60,11,0.000057
57,59,13,0.000067
56,50,17,0.000087
55,20,18,0.000092
54,52,19,0.000098
53,54,25,0.000128


In [27]:
zips = food_inspections_df['zip'].value_counts()
zips_df = pd.DataFrame({'zip':zips.index, 'count':zips.values})
zips_df['prevalence'] = zips_df['count'] / food_inspections_df.shape[0]

In [28]:
zips_df.nlargest(10, 'count')

,zip,count,prevalence
0,60614,7284,0.037414
1,60647,7088,0.036408
2,60657,6824,0.035052
3,60622,6108,0.031374
4,60611,6094,0.031302
5,60608,5925,0.030434
6,60618,5923,0.030424
7,60625,5387,0.027670
8,60639,5208,0.026751
9,60607,5145,0.026427


In [29]:
zips_df.nsmallest(10, 'count')

,zip,count,prevalence
60,60805,2,0.000010
61,60627,2,0.000010
59,nan,3,0.000015
58,60827,134,0.000688
57,60633,341,0.001752
56,60656,814,0.004181
55,60655,816,0.004191
54,60707,1125,0.005779
53,60604,1290,0.006626
52,60602,1409,0.007237


In [30]:
inspections = food_inspections_df['inspection_type'].value_counts()
inspections_df = pd.DataFrame({'inspection_type':inspections.index, 'count':inspections.values})
inspections_df['prevalence'] = inspections_df['count'] / food_inspections_df.shape[0]

In [31]:
inspections_df.nlargest(10, 'count')

,inspection_type,count,prevalence
0,Canvass,102944,0.528775
1,License,25614,0.131567
2,Canvass Re-Inspection,20457,0.105078
3,Complaint,18093,0.092935
4,License Re-Inspection,8899,0.045710
5,Complaint Re-Inspection,7499,0.038519
6,Short Form Complaint,6717,0.034502
7,Suspected Food Poisoning,850,0.004366
8,Consultation,669,0.003436
9,License-Task Force,604,0.003102


In [32]:
inspections_df.nsmallest(10, 'count')

,inspection_type,count,prevalence
41,TASK FORCE NOT READY,1,0.000005
42,No entry,1,0.000005
43,LICENSE,1,0.000005
44,CLOSE-UP/COMPLAINT REINSPECTION,1,0.000005
45,CITF,1,0.000005
46,Duplicated,1,0.000005
47,LICENSE CANCELED BY OWNER,1,0.000005
48,LIQOUR TASK FORCE NOT READY,1,0.000005
49,CANVAS,1,0.000005
50,HACCP QUESTIONAIRE,1,0.000005


In [33]:
results = food_inspections_df['result'].value_counts()
results_df = pd.DataFrame({'result':results.index, 'count':results.values})
results_df['prevalence'] = results_df['count'] / food_inspections_df.shape[0]

In [34]:
results_df.nlargest(10, 'count')

,result,count,prevalence
0,Pass,105369,0.541231
1,Fail,37658,0.193431
2,Pass w/ Conditions,26795,0.137633
3,Out of Business,16757,0.086073
4,No Entry,6198,0.031836
5,Not Ready,1843,0.009467
6,Business Not Located,64,0.000329


### Drop Violation Code for Now
We can join back using the Inspection ID to learn about types of violations, but we don't want to use any information about the violation itself to predict if a food inspection will pass or fail.

In [35]:
food_inspections_df = food_inspections_df.drop('violation_code', 1)

### Create Risk Group Feature
If the feature cannot be found in the middle of the text string as a value 1-3, return -1.

In [36]:
def create_risk_groups(risk_text):
    
    try: 
        risk = int(risk_text.split(' ')[1])
        return risk
    except:
        return -1

In [37]:
food_inspections_df['risk'] = food_inspections_df['risk'].apply(create_risk_groups)

### Format Result
- Encode Pass and Pass w/ Conditions as 0
- Encode Fail as 1
- Encode all others as -1 and filter out

In [38]:
def format_results(result):
    
    if result == 'Pass':
        return 0
    elif result == 'Pass w/ Conditions':
        return 0
    elif result == 'Fail':
        return 1
    else:
        return -1

In [39]:
food_inspections_df['result'] = food_inspections_df['result'].apply(format_results)
food_inspections_df = food_inspections_df[food_inspections_df['result'] != -1]

In [40]:
food_inspections_df.shape

(169822, 8)

### Filter for Categorical Features that Pass some Prevalence Threshold
This way we only consider fairly common attributes of historical food establishments and inspections so that our analysis will generalize to new establishments and inspections.

In [41]:
categorical_features = ['facility_type', 'zip', 'inspection_type']

In [42]:
def prev_filter(df, feature, prevalence='prevalence', prevalence_threshold=0.001):
    
    return df[df[prevalence] > prevalence_threshold][feature].tolist()

In [43]:
feature_dict = dict(zip(categorical_features, [prev_filter(facilities_df, 'facility_type'),
                                               prev_filter(zips_df, 'zip'),
                                               prev_filter(inspections_df, 'inspection_type')]))

### Encode Rare Features with the 'DROP' String, to be Removed Later
Note that by mapping all rare features to the 'DROP' attribute, we avoid having to one-hot-encode all rare features and then drop them after the fact.  That would create an unnecessarily large feature matrix.  Instead we one-hot encode features passing the prevalence threshold and then drop all rare features that were tagged with the 'DROP' string.

In [44]:
for feature in categorical_features:
    food_inspections_df[feature] = food_inspections_df[feature].apply(lambda x: 
                                                                      x if x in feature_dict[feature]
                                                                      else 'DROP')

In [45]:
feature_df = pd.get_dummies(food_inspections_df,
                            prefix=['{}'.format(feature) for feature in categorical_features],
                            columns=categorical_features)

In [46]:
feature_df = feature_df[[col for col in feature_df.columns if 'DROP' not in col]]

In [47]:
feature_df.shape

(169822, 96)

### Drop Features with:
- Risk level not recorded as 1, 2, or 3
- Result not recorded as Pass, Pass w/ Conditions, or Fail
- NA values (Some latitudes and longitudes were NA)

In [48]:
feature_df = feature_df[feature_df['risk'] != -1]
feature_df = feature_df[feature_df['result'] != -1]
feature_df = feature_df.dropna()

In [49]:
feature_df.shape

(169305, 96)

### Write the Feature Set to a Compressed CSV File to Load for Modeling and Analysis

In [50]:
feature_df.to_csv('../data/Food_Inspection_Features.gz', compression='gzip', index=False)

### Write off Zip Codes to Join with Census Data

In [51]:
zips_df.to_csv('../data/Zips.csv', index=False)